<a href="https://colab.research.google.com/github/rajendradayma/LLMs-/blob/main/Basic_Qwen_Chatbot_(Inference_Only).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install -q transformers gradio accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 12.2 MB/s eta 0:00:00


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import gradio as gr


In [4]:
model_name = "Qwen/qwen1.5-1.8B"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             torch_dtype=torch.float16,
                                             trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [11]:
def generate_response(message, history=[]):
    # Clean message
    message = message.strip()

    # ---🛡️ Input Checks---
    if len(message) > 500:
        return "⚠️ Input too long. Please shorten your message.", history

    # Too many zeros or meaningless repetition
    if message.count("0") > 100:
        return "⚠️ Too many repeating characters. Please ask a valid question.", history

    if not any(c.isalpha() for c in message):
        return "⚠️ Input must contain letters. Please rephrase your question.", history

    # ---📜 Build Prompt from History---
    prompt = ""
    for turn in history:
        prompt += f"[User]: {turn[0]}\n[Bot]: {turn[1]}\n"
    prompt += f"[User]: {message}\n[Bot]:"

    # ---🔍 Debug Print for Prompt---
    print("====== PROMPT SENT TO MODEL ======")
    print(prompt)
    print("==================================")

    # ---🧠 Generate Response---
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.7,
        do_sample=True,
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id,
    )

    # ---🔓 Decode Output---
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # ---✂️ Extract only the latest Bot response---
    bot_response = decoded.split("[Bot]:")[-1].strip()

    # Update conversation history
    history.append((message, bot_response))
    return bot_response, history


with gr.Blocks() as demo:
  gr.Markdown("## Qwen 1.5 chatbot demo ")
  chatbot = gr.Chatbot()
  msg = gr.Textbox(label="your Message")
  clear = gr.Button("clear chat")

  state = gr.State([])

  def user_input(user_input_text, history): # Renamed parameter to avoid conflict
    response, history = chat(user_input_text, history) # Calling the chat function
    return history, "", history

  msg.submit(user_input, [msg, state], [chatbot, msg, state])
  clear.click(lambda: None, None, chatbot, queue=False)

if __name__ == "__main__":
  demo.launch()

<ipython-input-11-3786352391>:51: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5a8d31fdecee03d9e0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
